In [12]:
import xesmf as xe
import xarray as xr
import numpy as np
import glob
import os
from netCDF4 import Dataset

In [8]:
def ocean_regrid(expname, varname):
    path_atm = glob.glob(f'/work/Katherine.Turner/tempest-extremes/{expname}/data/*.slp.nc')[0]
    path_atm
    ds_atm = xr.open_dataset(path_atm, use_cftime=True)

    ds_final_grid = xr.Dataset(
        {
            "lat": (["lat"], ds_atm.lat.values, {"units": "degrees_north"}),
            "lon": (["lon"], ds_atm.lon.values, {"units": "degrees_east"}),
        }
    )
    print("Output grid created...")

    files_to_regrid = sorted(glob.glob(f'/work/Katherine.Turner/tempest-extremes/{expname}/data/*.{varname}.nc'))
    
    for i in np.arange(len(files_to_regrid)):
        #check to see if regridded file exists
        file_out = files_to_regrid[i][:-3]+'.regrid.nc'
        file_out_tail = file_out.split('/data/')[-1]
    
        if os.path.isfile(file_out):
            print(f"{file_out_tail} exists. Continuing...")
        
        else:
            print(f"{file_out_tail} does not exist. Regridding...")
        
            ds_in = xr.open_dataset(files_to_regrid[i], use_cftime=True)
            da = ds_in[varname]
            regridder = xe.Regridder(ds_in, ds_final_grid, "bilinear", periodic=True)
            da_out = regridder(da)
            
            ds_out = da_out.to_dataset(name=varname)
            ds_out.to_netcdf(file_out)

In [9]:
expname = 'odiv-251'
varname = 'MLD_003'

ocean_regrid(expname,varname)

Output grid created...
ocean_daily_1x1deg.01510101-01551231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01560101-01601231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01610101-01651231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01660101-01701231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01710101-01751231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01760101-01801231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01810101-01851231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01860101-01901231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01910101-01951231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.01960101-02001231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.02010101-02051231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.02060101-02101231.MLD_003.regrid.nc exists. Continuing...
ocean_daily_1x1deg.02110101-02151231.MLD_003.regrid.n

In [13]:
def update_composite_calendar(expname,varname):

    filepath = f"/work/Katherine.Turner/tempest-extremes/{expname}/comp_{varname}_regrid/impulse_comp_{varname}_anom.nc"
    ds = Dataset(filepath, "a", format="NETCDF4")

    snap_time = ds.variables["snap_time"]
    snap_time.calendar = "noleap"
    ds.close()

In [14]:
update_composite_calendar("odiv-251", 'omldamax')